<a href="https://colab.research.google.com/github/a07458666/CrawlerAndBert/blob/master/dataMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from google.colab import drive
from tqdm import tqdm

#crawler
import requests
from bs4 import BeautifulSoup

In [ ]:
drive.mount('/content/drive')
os.chdir(r"/content/drive/My Drive/tbrainData/")
# 讀入官方的data
df = pd.read_csv('tbrain_train_final_0610.csv')  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wGEaf-w42sOYwCavbxMyiU6hUS2FmoOIOeSmQvj9dCipWNkvITnt-g
Mounted at /content/drive


In [ ]:
df[0:10]

,news_ID,hyperlink,content,name
0,1,https://news.cnyes.com/news/id/4352432,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,[]
1,2,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北...,[]
2,3,https://www.mirrormedia.mg/story/20191008soc011/,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,[]
3,4,https://www.chinatimes.com/realtimenews/201909...,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...,[]
4,5,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[]
5,6,http://www.coolloud.org.tw/node/93356,內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區 ### 省略內文 ### 市區拒...,[]
6,7,https://m.ctee.com.tw/livenews/gj/ctee/a076720...,華為創辦人任正非接受英國媒體訪問時表示 ### 省略內文 ### 重申中國和加拿大兩國之間的...,[]
7,8,https://www.mirrormedia.mg/story/20191209edi009/,2019.12.09 08:00「再教育營學員全已結業」新疆自治區主席：他們過上了幸福生活文...,[]
8,9,https://www.chinatimes.com/newspapers/20190827...,交通部觀光局26日公布7月國際旅客情況 ### 省略內文 ### 只能長期培養。,[]
9,10,https://news.cnyes.com/news/id/4365094,0理財基金基金汰弱留強該怎麼做？鉅亨買基金2019/08/06 21:30facebook ...,[]


In [ ]:
# 取出網站種類
webKind = df["hyperlink"].str.split('/').str.get(2)
webKind, count = np.unique(webKind, return_counts = True)
dict = {"name": webKind,  
        "counts": count
       }
webKind_df = pd.DataFrame(dict)
webKind_df

,name,counts
0,ccc.technews.tw,10
1,domestic.judicial.gov.tw,30
2,ec.ltn.com.tw,6
3,ent.ltn.com.tw,2
4,estate.ltn.com.tw,1
5,finance.technews.tw,56
6,hk.on.cc,57
7,house.ettoday.net,1
8,m.ctee.com.tw,530
9,m.ltn.com.tw,12


In [ ]:
def getUrlStatusCode(url):
  return requests.get(url).status_code

def getWebsiteCategory(url):
  return url.split('/')[2]

def getContext(url):
  Category = getWebsiteCategory(url)
  if getUrlStatusCode(url) != requests.codes.ok:
        return [], []
      
  # get the news
  res = requests.get(f'{url}')
  soup = BeautifulSoup(res.text)

  # get the title
  try:
    title = soup.find('h1').text
  except AttributeError as e:
    print("url : ", url, " get title fail")
    title = []

  # get contents   
  try:
    contents = soup.find_all('p')
  except:
    print("url : ", url, " get contents fail")
    return title, []

  lines = []
  for content in contents:
    try:
      content.find('a').txet
      continue
    except:
      pass
    oneLine = re.sub('[\(\)\{\}<>\n\t\xa0\r\u3000]', '', content.text)  
    lines.append(content.text)
  article = "".join(lines).strip()
  return title, article

In [ ]:
# 單獨測試
# 有問題 5
index = 6
url = df["hyperlink"][index]
category = getWebsiteCategory(url)
title, article = getContext(url) 
print("url :", url, "\ncategory :", category,'\ntitle', title, "\narticle :", article)

url : https://m.ctee.com.tw/livenews/gj/ctee/a07672002019081610333753 
category : m.ctee.com.tw 
title    工商時報 - 您投資理財的好夥伴   
article : 華為創辦人任正非接受英國媒體訪問時表示，他相信英國不會拒絕華為參與建設5G網路。此外，他也堅稱，華為設備並沒有安裝「後門」程式。任正非接受英國天空新聞（Sky News）訪問時指出，英國首相強森（Boris Johnson）是一名果斷且有能力的人，他上任3日後已表明英國應該盡快建設5G網絡，相信英國經過嚴格測試，並以認真態度研究後，不會拒絕華為。他重申，華為的電訊設備並無如外界傳聞般按中國政府要求安裝後門程式，形容華為員工不會做一些令華為「破產、自尋死路」的事。但他承認，美國政府將華為列入貿易黑名單，限制美國企業向華為出口零部件及軟體，對華為造成嚴重損害。任正非強調，自從美國針對華為以來，他從未與中共領導人見面，否則將跌入了美國總統川普的圈套，但說若非對方大肆「宣傳」華為，外界根本不知道華為產品的競爭力。針對加拿大政府應美國要求，逮捕華為副董事長孟晚舟，並可能被引渡到美國，任正非認為，這並非一件小事，但孟晚舟目前情況良好，正在如常生活，重申中國和加拿大兩國之間的政治問題不應將華為甚至其家人牽涉在內。


In [ ]:
df_addContext = df.copy()

In [ ]:
# 網站分類
#df_addContext['webKind'] = df_addContext['hyperlink'].apply(lambda x: getWebsiteCategory(x))

In [ ]:
#df_addContext['contextCrawler'] = df_addContext['hyperlink'].apply(lambda x: getContext(x))
contextList = []
hyperlinkList = df_addContext['hyperlink']
for index, url in enumerate(tqdm(hyperlinkList)):
    title, article = getContext(url)
    #print("url :", url, "\ncategory :", category,'\ntitle', title, "\narticle :", article)
    
    #contextList.append(context)

  0%|          | 1/5023 [00:01<1:43:18,  1.23s/it]

url : https://news.cnyes.com/news/id/4352432 
category : m.ctee.com.tw 
title 量化交易追求絕對報酬 有效對抗牛熊市 
article : 熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易備受注目。專家表示，採用量化交易策略投資台股，不管是處於多頭或是空頭市場，績效及波動度均可領跑大盤，甚至比國內投資台股的股票型基金及 ETF 的波動率還低，表現也更為穩定。大數據時代來臨，風行歐美 50 年的量化交易儼然成為顯學，台灣亦開始重視此一趨勢發展，也因此，中華機率統計學會及台北科技大學管理學院攜手主辦，並由元大期貨、摩根亞太量化交易等公司擔任協辦單位，今 (7/5) 日舉辦「時間序列與量化交易研討會」，就目前熱門的量化交易、智能投資等相關議題進行研討。量化交易的操作績效不亞於價值投資及技術投資，被譽為數學天才、最賺錢的基金經理人，文藝復興對沖基金創始人詹姆斯 · 西蒙斯 (James Simons) 所管理的大獎章 (Medallion) 基金，便是典型的量化交易，績效表現優異，不僅勝過索羅斯的量子基金，也打敗了股神巴菲特的價值投資。近年來台灣也逐漸重視量化交易，摩根亞太量化交易公司今年開始將量化交易引進台股投資，是國內首家推出量化交易策略的公司，初期對象鎖定法人機構，進行私募投資。張堯勇表示，數學不只是一門學科，更是一項扭轉乾坤、轉敗為勝的競爭利器，量化交易就是將數學運用在股市投資，透過複雜、精密的推理計算，打敗股市。他指出，目前全球利率水平 50 位於年來新低，美國十年期公債殖利率只有 2%，日本及一些歐洲國家甚至是負利率。歷史極低利率帶導致無風險及低風險工具的投資報酬率太低（例如：銀行存款、政府公債及投資等級公司債），無法對抗通貨膨脹及支付負債，所以資產配置必須增加較高風險的投資，如股票、高收益公司債或新興市場債券，但是這些投資工具波動很大，而且很容易產生虧損！張堯勇指出，量化投資具有低波動、絕對報酬、優於大盤的三大特點，摩根亞太量化交易模擬台股近 8 年來的績效，結果顯示，運用量化交易的年化報酬率及波動度均優於台股上市櫃指數，以 2011 年金融海嘯及 2018 年中美貿易戰為例，台股上市、上櫃指數均是負報酬，不過，摩根量化交易卻是正報酬，而且波

  0%|          | 2/5023 [00:03<2:07:17,  1.52s/it]

url : https://udn.com/news/story/120775/4112519 
category : m.ctee.com.tw 
title 外送員死亡未通報 北市勞動局開罰Uber Eats 30萬 
article : 10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件，北市府勞動局認定業者未依職業安全衛生法第37條第2項第1款規定於8小時內向勞動檢查機構通報，開罰Uber Eats 30萬元。


北市勞動局職安科科長康水順表示，職安衛部份已完成檢查，經訪談筆錄及取得的文件認定該名罹災勞工與Uber Eats具有僱傭關係，故依職業安全衛生法第37條第2項第1款，雇主應於8小時內通報，但至今無收到任何通報記錄；另外送員於發生職災死亡後，Uber Eats知悉事故的發生卻未通報，屬明知法令卻仍違法。

北市勞動局長賴香伶表示，由於平台業者標榜食物外送之快捷性，卻罔顧外送員行車安全，更以送達時間計算外送員工作獎金，致外送員為達平台業者規定，執行外送業務時於道路上高速行駛及穿梭於車陣，徒增行車事故風險。勞動局已事先向業者宣導法令，考量Uber Eats違法行為嚴重影響公共秩序，受責難程度高，依行政罰法、職業安全衛生法裁處罰鍰30萬元。

勞動局職安科表示，今日先針對職災通報部分裁處作成決議，後續業者違反勞基法的部分，將根據違法部分另行開罰。除針Uber Eats黃姓外送人員職災死亡卻未依法通報案件開罰業者30萬元，北市政府今日下午也跨局處召開「外送人員保障專案會議」。圖／台北市勞動局提供
                    外送員車禍又一樁！Uber Eats林姓外送員與柯姓女駕駛發生碰撞，林男身體擦傷，送醫後無礙。近日外送平台崛起，路上可見...                  
                    新北市府交通局今天表示，將規劃機車卸貨停車區域，並開放外送員臨時停靠家長接送區，公有停車場也提供機車免費停車時段，方便機...                  
                    近來外送產業爭議不斷，台北市政府警察局今邀集交通局及Uber eats、Foodpanda、Lalamove等6家外送平...                  
                    現在外送

  0%|          | 3/5023 [00:08<3:32:26,  2.54s/it]

...

  0%|          | 23/5023 [00:43<2:40:07,  1.92s/it]

url : http://news.ltn.com.tw/news/society/breakingnews/2920533 
category : m.ctee.com.tw 
title 倒「ㄆㄨㄣ」又揹農藥桶噴不明液體 惡鄰恐嚇鄰居判刑2月 
article : 為達最佳瀏覽效果，建議使用 Chrome、Firefox 或 Microsoft Edge 的瀏覽器。爆吳男揹著農藥桶噴霧桶噴鄰居。示意圖。（資料照）〔記者王定傳／新北報導〕52歲吳姓男子因裝潢施工問題與鄰居相處不睦，竟連續3天在樓梯間倒廚餘，他因此被罰1萬元後，仍不知悔改，改揹農用噴灑器，朝陳姓與黃姓住戶噴灑不明液體，其中一名被害者至今仍會做惡夢，吳男否認犯行，新北地院不採信，依恐嚇危害安全罪，判他2個月徒刑，可上訴。檢警調查，2017年底，吳男買下新北市永和區某頂樓舊公寓後，想裝潢成套房出租，因裝潢工程未經許可被鄰居檢舉；去年5月，他為報復鄰居，連續3天在1至4樓樓梯間傾倒廚餘，法院認定吳男所為已違反社會秩序維護法中的「藉端滋擾住戶」裁罰1萬元。請繼續往下閱讀...吳男仍不知悔改，去年10月23日，改揹農用噴灑器，朝著陳、黃兩位鄰居噴灑不明液體，2人嚇傻，其中一人至今仍會因此做惡夢，整個晚上都無法入眠，另一人則說很害怕吳男噴灑農藥，擔心生命或健康受到危險。檢方偵辦時，吳男辯稱，噴灑的不明液體只是水，由於檢方無法證明噴出的不明液體對人體有害，僅能依恐嚇危害安全罪嫌起訴；審理時，他仍辯稱沒有噴灑不明液體，亦無恐嚇之意。不過，除被害者指證歷歷外，法官還查出，吳男之前就曾與不明男子在樓梯間噴灑農藥，另外，勘驗案發時監視器畫面可知，陳姓女住戶當時背著幼童走下樓梯，吳男卻朝她及空中高舉噴灑器噴頭，被害者立即以手遮住幼童口鼻並折返，吳男仍持續朝她的方向及空中高舉噴灑器噴頭。法官判定，吳男藉由噴灑不明液體方式恐嚇住戶，顯見其自我情緒管理能力及尊重他人法益法治觀念，均待加強，犯罪後矢口否認犯行，未與被害者和解，判刑2月，可上訴。
                    桃市交警進駐武陵、青溪及龍安3派出所 優化交通事故處理效能                                    
                    9/8 今彩539開出3注！ 獎落台北文山、中山區              

  0%|          | 24/5023 [00:47<3:12:39,  2.31s/it]

url : https://udn.com/news/story/6809/4188069 
category : m.ctee.com.tw 
title 華郵：壞消息不斷 習近平焦頭爛額 
article : 華盛頓郵報報導，壞消息引發外界揣測中國領導班子承受巨大內部壓力，尤其時值美中貿易戰似難很快獲得解決。
習近平雖取消國家主席的任期限制，讓自己能無限連任，但不像能完全掌握住局勢，儘管他透過宣傳「習近平思想」，大搞個人崇拜，並大舉打貪反腐導致逾150萬官員落馬。
聚焦中國事務的內參部落格共同主編姜雲（Yun Jiang，音譯）說：「體系裡洩密的事相當罕見，但並不代表領導階層處於危機或團結弱化。」
然而，事態發展仍不利中國。關於新疆，顯然至少有一名高階官員打抱不平，甘冒風險將資料交給外國記者──最初提供給紐約時報，後來又有國際調查記者聯盟經手。這些資料顯示中國想漢化西部地區維吾爾穆斯林少數民族，整個體系由習近平親自下令。
澳洲共諜案較為複雜。王立強自稱替中國軍情單位滲透香港，負有干預台灣2020年大選、推翻蔡英文的任務，澳洲表示會認真看待他的說詞，但部分專家指前後矛盾多，更別提假護照製作太外行。
在香港，經過6個月示威後，中國仍對林鄭月娥表達支持。但有人認為港府應提早採取更嚴厲的行動鎮壓示威。天津南開大學教授李曉兵說：「香港在發高燒，選舉是反映香港社會體溫的溫度計。港府的作為相當溫和，早該採取行動鎖定反對派，卻沒有這樣做，反而讓反對勢力有機會『蹬鼻子上臉』」。
儘管事有湊巧，但一連串事件剛好符合中國對於西方強權試圖阻礙其和平崛起的說法。中方官媒的報導充斥有關美國中情局和其他西方情報單位「伸黑手」，煽動香港動盪，以便施壓北京的指控。
現在問題是：北京會如何回應？在北大深圳校區任教九年的經濟學教授教鮑爾丁說：「歷史若能為鑒，情況不會太好。不管是持續裝襲作啞或更嚴厲的鎮壓，他們似乎格外無力對既定計畫進行任何調整。」

                    加拿大卑詩省最高法院21日宣布，法官下周將公布中國大陸電信巨頭華為財務長孟晚舟是否構成「雙重犯罪」的判決結果，這是孟晚舟...                  
                    阿富汗即將與民兵組織塔利班展開和平談判，但政府官員表示，首都喀布爾市中心今天發生爆炸，副總統沙雷車隊遭鎖定攻擊，

KeyboardInterrupt: ignored

In [ ]:
df_addContext["content_c"] = ""
for index, text in enumerate(tqdm(contextList)):
    df_addContext.at[index, "content_c"] = text

100%|██████████| 5023/5023 [00:00<00:00, 125713.74it/s]


In [ ]:
df_addContext.to_csv('./data/Result.csv')